In [1]:
import pandas as pd
from datetime import date, datetime

## From IEX Data

In [2]:
from iexfinance.stocks import get_historical_data

startdate = datetime(2015, 1, 1)
enddate = date.today()
iex_aapl = get_historical_data('AAPL', startdate, enddate, output_format='pandas')

iex_aapl.head()

,open,high,low,close,volume
date,,,,,
2015-01-02,103.4410,103.4874,99.6893,101.5280,53204626
2015-01-05,100.5622,100.8965,97.8877,98.6678,64285491
2015-01-06,98.9371,99.7636,97.1634,98.6771,65797116
2015-01-07,99.5500,100.4786,99.0810,100.0607,40105934
2015-01-08,101.4351,104.1468,100.9429,103.9053,59364547


In [3]:
def get_iex_dict(symbol_list, startdate):
    """Get dictionary of dataframes from IEX"""
    dict_list = dict()
    startdate = datetime.strptime(startdate, '%Y-%m-%d')
    for symbol in symbol_list:
        df = get_historical_data(symbol, startdate, date.today(), output_format='pandas')
        dict_list[symbol] = df
    return dict_list

symbol_list = ['AAPL', 'AMZN', 'NFLX', 'INTC', 'SPY', 'QQQ']
iex_dict = get_iex_dict(symbol_list, '2015-01-01')
iex_dict['AMZN'].tail()

,open,high,low,close,volume
date,,,,,
2019-04-12,1848.40,1851.50,1841.30,1843.06,3114413
2019-04-15,1842.00,1846.85,1818.90,1844.87,3724423
2019-04-16,1851.35,1869.77,1848.00,1863.04,3044618
2019-04-17,1872.99,1876.47,1860.44,1864.82,2893517
2019-04-18,1868.79,1870.82,1859.48,1861.69,2749882


In [4]:
from iexfinance.stocks import get_historical_intraday

iex_aapl_itd = get_historical_intraday("AAPL", startdate='2019-04-01', enddate='2019-04-18',  output_format='pandas')
print(iex_aapl_itd.tail(15))

                     average  changeOverTime    close      date     high  \
2019-04-18 15:45:00  203.828        0.003056  203.850  20190418  203.855   
2019-04-18 15:46:00  203.846        0.003145  203.840  20190418  203.860   
2019-04-18 15:47:00  203.848        0.003154  203.880  20190418  203.880   
2019-04-18 15:48:00  203.832        0.003076  203.835  20190418  203.890   
2019-04-18 15:49:00  203.819        0.003012  203.790  20190418  203.860   
2019-04-18 15:50:00  203.826        0.003046  203.880  20190418  203.920   
2019-04-18 15:51:00  203.763        0.002736  203.700  20190418  203.870   
2019-04-18 15:52:00  203.786        0.002849  203.780  20190418  203.820   
2019-04-18 15:53:00  203.801        0.002923  203.710  20190418  203.930   
2019-04-18 15:54:00  203.714        0.002495  203.695  20190418  203.760   
2019-04-18 15:55:00  203.682        0.002338  203.765  20190418  203.765   
2019-04-18 15:56:00  203.789        0.002864  203.870  20190418  203.880   
2019-04-18 1

## AlphaVantage (HTTP Request)

In [9]:
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize

url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=2318.HK&apikey=[yourAPIKey]'
response = urlopen(url)
json_2318 = response.read()

data_2318 = json.loads(json_2318)
df_2318 = pd.DataFrame.from_dict(data_2318['Time Series (Daily)'])
df_2318 = df_2318.T
df_2318.index = pd.to_datetime(df_2318.index)
print(df_2318.head())

            1. open  2. high   3. low 4. close 5. volume
2019-04-18  95.6500  95.6500  94.1000  94.5000  24229379
2019-04-17  94.7500  95.6500  93.7000  94.4500  30245080
2019-04-16  92.5000  95.1000  92.1500  94.7500  35753933
2019-04-15  93.9500  95.2500  92.5500  92.6000  40867262
2019-04-12  91.7000  92.6500  91.3000  92.6500  23123193


In [11]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=QQQ&interval=5min&apikey=[yourAPIKey]'
response = urlopen(url)
json_qqq = response.read()

data_qqq = json.loads(json_qqq)
df_qqq = pd.DataFrame.from_dict(data_qqq['Time Series (5min)'])
df_qqq = df_qqq.T
df_qqq.index = pd.to_datetime(df_qqq.index)
print(df_qqq.head())



                      1. open   2. high    3. low  4. close 5. volume
2019-04-18 16:00:00  187.2400  187.4100  187.1900  187.3800   1039722
2019-04-18 15:55:00  187.2800  187.3400  187.1700  187.2500   1273252
2019-04-18 15:50:00  187.2600  187.3000  187.2100  187.2800    402529
2019-04-18 15:45:00  187.2800  187.3100  187.2100  187.2600    202288
2019-04-18 15:40:00  187.3700  187.3800  187.2237  187.2779    436000


## Tushare (For Chinese A & B Shares)

### Individual stock EOD data

In [5]:
import tushare as ts

pro = ts.pro_api('api-token')
df_0036 = pro.daily(ts_code='600036.SH', start_date='20140101', end_date='20190418')
print(df_0036.head(10), '\n')
print(df_0036.tail(10))

     ts_code trade_date   open   high    low  close  pre_close  change  \
0  600036.SH   20190418  35.80  35.80  35.13  35.50      35.83   -0.33   
1  600036.SH   20190417  35.89  36.48  35.63  35.83      36.00   -0.17   
2  600036.SH   20190416  34.70  36.10  34.60  36.00      34.82    1.18   
3  600036.SH   20190415  35.42  36.22  34.82  34.82      34.80    0.02   
4  600036.SH   20190412  34.86  35.00  34.60  34.80      34.96   -0.16   
5  600036.SH   20190411  35.19  35.65  34.72  34.96      35.07   -0.11   
6  600036.SH   20190410  35.05  35.43  34.52  35.07      35.23   -0.16   
7  600036.SH   20190409  35.90  35.91  35.06  35.23      35.93   -0.70   
8  600036.SH   20190408  35.70  36.27  35.65  35.93      35.59    0.34   
9  600036.SH   20190404  34.90  36.05  34.90  35.59      34.86    0.73   

   pct_chg        vol       amount  
0  -0.9210  322740.70  1143221.193  
1  -0.4722  494789.82  1778194.094  
2   3.3889  840815.16  2991158.275  
3   0.0575  924920.92  3299629.416  


### Print list of all Shanghai-Connect stocks

In [7]:
pro = ts.pro_api('2856ca4d686325824d126f59b0fe5cac71128847e713af9d6d156671')
df_components = pro.hs_const(hs_type='SH')
print(df_components.head(10), '\n')
print(df_components.tail(10))

     ts_code hs_type   in_date out_date is_new
0  603818.SH      SH  20160613     None      1
1  603108.SH      SH  20161212     None      1
2  600507.SH      SH  20141117     None      1
3  601377.SH      SH  20141117     None      1
4  600309.SH      SH  20141117     None      1
5  600298.SH      SH  20141117     None      1
6  600018.SH      SH  20141117     None      1
7  600483.SH      SH  20151214     None      1
8  600068.SH      SH  20141117     None      1
9  600594.SH      SH  20141117     None      1 

       ts_code hs_type   in_date out_date is_new
567  601015.SH      SH  20150615     None      1
568  600557.SH      SH  20141117     None      1
569  603118.SH      SH  20151214     None      1
570  600988.SH      SH  20150615     None      1
571  603306.SH      SH  20150521     None      1
572  601000.SH      SH  20141117     None      1
573  600171.SH      SH  20141117     None      1
574  600422.SH      SH  20141117     None      1
575  601991.SH      SH  20141117     Non

### From STOOQ (for Futures & Forex)

In [15]:
import os
import time
from selenium import webdriver
browser = webdriver.Chrome(os.path.join('chromedriver.exe'))

import shutil
download_path = os.path.join(os.path.expanduser('~'), 'Downloads')

def get_stooq_move(symbol_list, download_path, csv_path, type):
    dict_link = {'indices': '', 'futures': '.F', 'forex': ''}
    dict_csv = {'indices': '_d', 'futures': '_f_d', 'forex': '_d'}
    for ticker in symbol_list:
        # Use Selenium Chrome to click to download CSV (urlretrieve NOT working well)
        url = 'https://stooq.com/q/d/?s=' + ticker + dict_link[type]
        browser.get(url)
        link_csv = browser.find_element_by_link_text('Download data in csv file...')
        link_csv.click()
        time.sleep(1)
        # Move the file from download folder to own target folder
        target_file = os.path.join(csv_path, ticker + '_' + date.today().strftime('%Y-%m-%d') + '.csv')
        shutil.move(os.path.join(download_path, ticker.lower() + dict_csv[type] + '.csv'), target_file)
    browser.close()

csv_path = os.path.join(os.path.abspath('csv_STOOQ'))
if not os.path.exists(csv_path):
    os.makedirs(csv_path)

indices_list = ['^HSI', '^SPX', '^DJI', '^DAX', '^NKX']
futures_list = ['ES', 'NQ', 'YM', 'RJ', 'GC', 'CL']
forex_list = ['EURUSD', 'GBPUSD', 'USDJPY', 'USDCAD', 'EURJPY', 'GBPJPY']

get_stooq_move(indices_list, download_path, csv_path, 'indices')
get_stooq_move(futures_list, download_path, csv_path, 'futures')
get_stooq_move(forex_list, download_path, csv_path, 'forex')


In [18]:
df_es = pd.read_csv(os.path.join(csv_path, '^spx_' + date.today().strftime('%Y-%m-%d') + '.csv'), header=0, index_col='Date', parse_dates=True)
df_es.loc['1987-10-19':].head()

,Open,High,Low,Close,Volume
Date,,,,,
1987-10-19,282.70,282.70,224.83,224.84,671444433.0
1987-10-20,225.06,245.62,216.46,236.83,675666679.0
1987-10-21,236.83,259.27,236.83,258.38,499555556.0
1987-10-22,258.24,258.38,242.99,248.25,435777801.0
1987-10-23,248.29,250.70,242.76,248.22,272888889.0
